# MFMViT-QF Training
Training script for MFMViT with FBCNN QF conditioning

## 1. Imports

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import os

from models.mfmQF import MFMViT_QF
from src.fft_utils import apply_fft, apply_ifft, get_mask, get_spectrum_amplitude
from src.loss import MFMLoss
from src.qf_extraction import load_fbcnn_qf

c:\Users\admin\miniconda3\envs\deepfake-detection\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not load this library: C:\Users\admin\miniconda3\envs\deepfake-detection\Lib\site-packages\torchvision\image.pyd'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 2. Configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Training config
img_size = 224
batch_size = 16
lr = 1e-4
epochs = 50
mask_ratio = 0.5
qf_dim = 64

Using device: cuda


## 3. Data Loading

In [3]:
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('E:/data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = datasets.ImageFolder('E:/data/val', transform=transform) if os.path.exists('E:/data/val') else None
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) if val_dataset else None

print(f"Train samples: {len(train_dataset)}")
if val_dataset:
    print(f"Val samples: {len(val_dataset)}")

Train samples: 80216
Val samples: 6197


## 4. Model Setup

In [4]:
# Load frozen FBCNN for QF extraction
fbcnn_qf = load_fbcnn_qf('checkpoints/FBCNN/fbcnn_color.pth', qf_dim=qf_dim, device=device)

# Initialize MFMViT-QF model
model = MFMViT_QF(img_size=img_size, qf_dim=qf_dim, use_qf=True).to(device)

# Optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = MFMLoss().to(device)

# Create checkpoint directory
os.makedirs('checkpoints/MFMQF', exist_ok=True)

print("Models initialized successfully!")

[FBCNN] Loaded pretrained weights from checkpoints/FBCNN/fbcnn_color.pth
[FBCNN] Frozen with qf_dim=64
Models initialized successfully!


## 5. Training Loop

In [5]:
for epoch in range(epochs):
    model.train()
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
    train_loss = 0
    
    for images, _ in pbar:
        images = images.to(device)
        
        # Extract QF vectors using FBCNN
        with torch.no_grad():
            qf_vectors = fbcnn_qf(images)
        
        # FFT processing
        fft_original = apply_fft(images)
        amplitude_original = get_spectrum_amplitude(fft_original)
        
        mask = get_mask(images.shape[0], images.shape[1], img_size, 
                      ratio=mask_ratio, device=device)
        fft_masked = fft_original * mask
        
        corrupted_spatial = apply_ifft(fft_masked)
        predicted_spatial = model(corrupted_spatial, qf_vectors)
        
        fft_predicted = apply_fft(predicted_spatial)
        amplitude_predicted = get_spectrum_amplitude(fft_predicted)
        
        loss = criterion(amplitude_predicted, amplitude_original, mask)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # Validation
    val_loss = 0
    if val_loader:
        model.eval()
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                qf_vectors = fbcnn_qf(images)
                
                fft_original = apply_fft(images)
                amplitude_original = get_spectrum_amplitude(fft_original)
                
                mask = get_mask(images.shape[0], images.shape[1], img_size, 
                              ratio=mask_ratio, device=device)
                fft_masked = fft_original * mask
                
                corrupted_spatial = apply_ifft(fft_masked)
                predicted_spatial = model(corrupted_spatial, qf_vectors)
                
                fft_predicted = apply_fft(predicted_spatial)
                amplitude_predicted = get_spectrum_amplitude(fft_predicted)
                
                loss = criterion(amplitude_predicted, amplitude_original, mask)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
    
    train_loss /= len(train_loader)
    msg = f'Epoch {epoch+1} | Train Loss: {train_loss:.4f}'
    if val_loader:
        msg += f' | Val Loss: {val_loss:.4f}'
    print(msg)
    
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f'checkpoints/MFMQF/mfm_qf_epoch_{epoch+1}.pth')

torch.save(model.state_dict(), 'checkpoints/MFMQF/mfm_qf_final.pth')
print('Training complete!')

Epoch 1/50:   0%|          | 0/5014 [00:00<?, ?it/s]

[MFMViT-QF] QF modulation enabled: qf_dim=64 -> embed_dim=768
[MFMViT-QF init] embed_dim=768, num_patches=196, patches_per_side=14, patch_size=16, patch_dim=768


Epoch 1/50: 100%|██████████| 5014/5014 [14:51<00:00,  5.62it/s, loss=0.7825]


Epoch 1 | Train Loss: 0.7889 | Val Loss: 0.7506


Epoch 2/50: 100%|██████████| 5014/5014 [14:27<00:00,  5.78it/s, loss=0.7137]


...

Epoch 48 | Train Loss: 0.2129 | Val Loss: 0.2128


Epoch 49/50: 100%|██████████| 5014/5014 [14:34<00:00,  5.74it/s, loss=0.1955]


Epoch 49 | Train Loss: 0.2117 | Val Loss: 0.2126


Epoch 50/50: 100%|██████████| 5014/5014 [14:34<00:00,  5.73it/s, loss=0.2278]


Epoch 50 | Train Loss: 0.2106 | Val Loss: 0.2115
Training complete!
